In [1]:
!git clone https://github.com/grayfactory/Toxicity-Under-Context-Detect.git
%cd Toxicity-Under-Context-Detect/
!ls

Cloning into 'Toxicity-Under-Context-Detect'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 31 (delta 10), reused 28 (delta 7), pack-reused 0
Unpacking objects: 100% (31/31), done.
Checking out files: 100% (14/14), done.
/content/Toxicity-Under-Context-Detect
data	   Preprocessing_data_and_prepare4training.ipynb  utils
model_ckp  README.md


In [3]:
!pip install transformers
!pip install datasets

from IPython.display import clear_output 

from transformers import (ElectraTokenizer, ElectraForSequenceClassification,
                          AdamW, get_linear_schedule_with_warmup)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F


import pandas as pd
import numpy as np
from pathlib import Path
import random
import time

from utils._utils import MyConfig, plot_confusion_matrix, plot_history
from utils.model import (tokenize_segmented_text, TOXICDataset, 
                         f1_score_model, AverageMeter, 
                         train_epoch, eval_model, model_score, EarlyStopping)


clear_output()

In [ ]:
# for load model
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp'),
    'path_colab_data' : Path('./data')
    },
    'seeds' : 42,
    'model_to_use' : 'google/electra-small-discriminator',
    'tokenizer_to_use' : 'google/electra-small-discriminator',
    'model_ckp_dir' : Path('/content/gdrive/MyDrive/toxic_nlp/Electra-model-save')
})
configs.path['list_csv'] = [f for f in (configs.path['path_colab_data']).glob('*.csv')]

if configs.seeds:
  random.seed(configs.seeds)
  np.random.seed(configs.seeds)
  torch.manual_seed(configs.seeds)


# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
if torch.cuda.is_available():
  configs.device = "cuda"
else: configs.device="cpu"

print("Using {} device".format(configs.device))


if configs.model_ckp_dir.exists():
  configs.source = 'gdrive'
else : 
  configs.model_ckp_dir = Path('./model_ckp/')
  configs.source = 'github'

configs

Using cpu device


{'device': 'cpu',
 'model_ckp_dir': PosixPath('model_ckp'),
 'model_to_use': 'google/electra-small-discriminator',
 'path': {'list_csv': [PosixPath('data/combination-gan-gan.csv'),
   PosixPath('data/combination-ori-gan.csv'),
   PosixPath('data/gc_clean.csv'),
   PosixPath('data/combination-gan-ori.csv'),
   PosixPath('data/combination-all-merged.csv'),
   PosixPath('data/combination-ori-ori.csv'),
   PosixPath('data/jig_test.csv'),
   PosixPath('data/jig_all.csv')],
  'path_colab_data': PosixPath('data'),
  'path_drive': PosixPath('/content/gdrive/MyDrive/toxic_nlp')},
 'seeds': 42,
 'source': 'github',
 'tokenizer_to_use': 'google/electra-small-discriminator'}

In [5]:
##
## train_condition setting
##
configs.train_condition = 'ori-ori'
configs.model_save_dir = configs.path['path_drive'] / f'Electra-model-save/electra-{configs.train_condition}.pt'
configs.max_len = 256

# incase evaluation with uploaded model
if configs.source == 'github':
  configs.model_save_dir = configs.model_ckp_dir / 'electra-generated-data-full-merged.pt'


In [8]:
def get_data_csv_path(configs, condition):
  condition = condition
  return [f for f in configs.path['list_csv'] if f'{condition}' in str(f)][0]

df_gan = pd.read_csv(get_data_csv_path(configs, configs.train_condition), index_col=0)
df_gan

,text,parent,label
0,"BTW, I see no ""attack"" here. What I see is som...",I also feel its important for StuRat to have h...,0
1,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0
2,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0
3,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the ...,0
4,I like the additional info in the info box. It...,The larger box displays fine in Mozilla Firefo...,0
...,...,...,...
23669,I guess 'cause there are some checks keeping t...,"That's almost too ridiculous to be true, but I...",1
23670,"Don’t cry too hard into your waifu, faggot!",You sound triggered. Did your sippy cup run ou...,1
23671,The media will cover for the Dems and call rep...,Here's what's going to happen: Broward is magi...,0
23672,"I dont know exactly how you feel, of course, b...","Yeah tone ""ghost"" from the victory inn is stil...",1


In [9]:
# sampling random data for inference
df_time = df_gan.sample(frac=0.02, random_state=configs.seeds)
df_time.shape, df_time.label.value_counts()


((473, 3), 1    242
 0    231
 Name: label, dtype: int64)

In [17]:
configs.model_save_dir

PosixPath('model_ckp/electra-generated-data-full-merged.pt')

In [10]:
# fine-tuning model with condition!
model_generated = ElectraForSequenceClassification.from_pretrained(configs.model_to_use)
model_generated.load_state_dict(torch.load(configs.model_save_dir, map_location=torch.device('cpu')))

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

<All keys matched successfully>

In [11]:
tokenizer = ElectraTokenizer.from_pretrained(configs.tokenizer_to_use)

In [12]:
df_gan.iloc[0,0]

'BTW, I see no "attack" here. What I see is something I might describe as "a group of editors discussing problematic editing, and asking for a change in behavior". It\'s seeing this as an "attack" and making things into a personal battle that is the \'problem\' here. More of the same is \'not the solution\'.'

In [13]:
def encoding_to_tensor(text):
  encoding = tokenize_segmented_text(text[0], text[1], tokenizer, configs.max_len)
  label = text[2]

  return { # 중요!! 여기서는 dataloader를 사용하지 않기 때문에, batch 차원을 넣어주어야 한다.
        'input_ids' : torch.tensor(encoding['input_ids'], dtype=torch.long).unsqueeze(0), 
        'attention_mask' : torch.tensor(encoding['attention_mask'], dtype=torch.float32).unsqueeze(0),
        'token_type_ids' : torch.tensor(encoding['segment_ids'], dtype=torch.long).unsqueeze(0),
        'label' : torch.tensor(label, dtype=torch.long)
    }

# test
encoding = encoding_to_tensor(df_gan.iloc[0,:])
y_pred = model_generated(input_ids = encoding['input_ids'],
                         attention_mask = encoding['attention_mask'],
                         token_type_ids = encoding['token_type_ids'])[0]
prob, pred = torch.max(F.softmax(y_pred, dim=1), dim=1)
prob.item(), pred.item()





(0.9938370585441589, 0)

In [ ]:
for text, parent, label in zip(df_time.text,df_time.parent,df_time.label):
  print(text)

In [15]:
inference_time_list = []
for text, parent, label in zip(df_time.text,df_time.parent,df_time.label):

  
  enc_time = time.time()
  encoding = encoding_to_tensor((text, parent, label))
  enc_end = time.time()

  start_time = time.time()
  y_pred = model_generated(input_ids = encoding['input_ids'],
                          attention_mask = encoding['attention_mask'],
                          token_type_ids = encoding['token_type_ids'])[0]
  prob, pred = torch.max(F.softmax(y_pred, dim=1), dim=1)
  end_time = time.time()

  inference_time = end_time - start_time
  encod_time = enc_end - enc_time
  inference_time_list.append(encod_time+inference_time)
  # print(prob.item(), pred.item(), encod_time, inference_time)



In [16]:
inference_t = sum(inference_time_list) / len(inference_time_list)
print(f'Model Inference Tiem : {inference_t:.3}s')

Model Inference Tiem : 0.15s
